In [1]:
from pickle import dump, load
v='07.12.23'

with open(f'../pkl/union.db.all.go.{v}.pkl','rb') as f:
    union_fa_all_upd = load(f)
with open('../pkl/IA.dict.pkl','rb') as f:
    IAdict = load(f)
with open(f'../pkl/pdb.all.go.{v}.v2.pkl','rb') as f:
    pdb_goa_upd = load(f)

merged_goa = union_fa_all_upd | pdb_goa_upd

In [16]:
merged_goa['Q9LYY7']

[]

In [17]:
with open('../pkl/important.prots.v2.pkl','rb') as f:
    important_proteins = load(f)

In [18]:
import csv
from pickle import dump, load
from tqdm import tqdm
v='uniprot07.pdb08'
topN = 3

prost_res = {}
with open(f'{v}.05.prost.res.tsv','r') as f:
    for line in f:
        line = line.strip().split('\t')
        q = line[0]
        h = line[2]
        if h not in merged_goa: continue
        if q not in prost_res: prost_res[q] = []
        if len(prost_res[q]) < topN: prost_res[q].append(h)

In [20]:
#filter out go terms with IA = 0
#merge the go terms from homologs, but dont mix the ranking.
with open(f'{v}.prost.e05.top{topN}.small.tsv','w') as f:
    for p in important_proteins:
        if p not in prost_res: continue
        goMap = {}
        for i,h in enumerate(prost_res[p]):
            score = 1 - i*(1/topN)
            for go in merged_goa[h]:
                if go not in IAdict: continue
                if IAdict[go] == 0: continue
                if go not in goMap: goMap[go] = score
        for go in goMap:
            f.write(f'{p}\t{go}\t{goMap[go]}\n')

In [21]:
!tar -czf uniprot07.pdb08.prost.e05.top3.small.tar.gz uniprot07.pdb08.prost.e05.top3.small.tsv